# PyTorch Estimator to run a script to train

In [ ]:
from aistudio.pytorch import PyTorch

script_params = {
    '--batch-size': 64,
    '--test-batch-size': 128,
    '--lr': 0.5,
    '--epochs': 10
    #,'--nprocs': 4
}
estimator = PyTorch(script="train.py",script_params=script_params)

In [ ]:
estimator.fit()

# Estimator with Model File(.onnx)

estimator needs test-batch-size, test dataloader and loss function

In [ ]:
from aistudio.estimator import Estimator
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch

script_params = {
    '--test-batch-size': 128
}

from torchvision import datasets, transforms
download_root = 'test-data'
mnist_transform=transforms.Compose([
                           transforms.ToTensor()
                           ,transforms.Normalize((0.1307,), (0.3081,))
                           ])
test_dataset  = datasets.MNIST(download_root, transform=mnist_transform, train=False, download=True)
# Data Preperation
input_data = test_dataset.data
input_data = torch.add(input_data,-0.1307)
input_data = torch.div(input_data,0.3081)
input_data = input_data.unsqueeze(1)
input_labels = test_dataset.targets

estimator = Estimator(use_model=True,model_path="mnist_net.onnx",script_params=script_params)
estimator.validate(input_data, input_labels, loss="nll_loss")

# Estimator with Model File(.pth)

Estimator with PyTorch Model File

In [1]:
from aistudio.estimator import Estimator
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

# define script parameters
script_params = {
    '--test-batch-size': 128
}
# Data Preparation and Pre-Processing
download_root = 'test-data'
mnist_transform=transforms.Compose([
                           transforms.ToTensor()
                           ,transforms.Normalize((0.1307,), (0.3081,))
                           ])
test_dataset  = datasets.MNIST(download_root, transform=mnist_transform, train=False, download=True)

### Data Pre-processing for testset

In [2]:
import torch

input_data = test_dataset.data
input_data = torch.add(input_data,-0.1307)
input_data = torch.div(input_data,0.3081)
input_data = input_data.unsqueeze(1)
input_labels = test_dataset.targets

### Create Network

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

## Validate(Test) Model

In [7]:
net = Net()
estimator = Estimator(use_model=True,model_path="mnist_net.pth",network=net,script_params=script_params)
estimator.validate(input_data, input_labels, loss="nll_loss")

PyTorch Model was Loaded.
mnist_net .pth
Validation by PyTorch Model.
Loss: 8.533471	Accuracy: 98.39


## Fit Model

#### Data Preparation and Preprocessing for training set

In [3]:
download_root = 'train-data'
mnist_transform=transforms.Compose([
                           transforms.ToTensor()
                           ,transforms.Normalize((0.1307,), (0.3081,))
                           ])
train_dataset  = datasets.MNIST(download_root, transform=mnist_transform, train=True, download=True)
input_data = train_dataset.data
input_data = torch.add(input_data,-0.1307)
input_data = torch.div(input_data,0.3081)
input_data = input_data.unsqueeze(1)
input_labels = train_dataset.targets

NameError: name 'torch' is not defined

### to use only model file(it couldn't update weights)

In [9]:
script_params = {
    '--epochs': 5,
    '--batch-size': 64,
    '--test-batch-size': 128,
    '--lr': 0.05
}

net = Net()
estimator = Estimator(use_model=True,model_path="mnist_init_net.pt",network=net,script_params=script_params,debug=True)
estimator.fit(input_data, input_labels, loss="nll_loss", opt="SGD")

PyTorch Model was Loaded.
Model's state_dict:
conv1.weight 	 torch.Size([10, 1, 5, 5])
conv1.bias 	 torch.Size([10])
conv2.weight 	 torch.Size([20, 10, 5, 5])
conv2.bias 	 torch.Size([20])
fc1.weight 	 torch.Size([50, 320])
fc1.bias 	 torch.Size([50])
fc2.weight 	 torch.Size([10, 50])
fc2.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.05, 'momentum': 0.5, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139750923564288, 139750923580512, 139750923581632, 139750923582192, 139750923580752, 139750923518352, 139753063290272, 139753063334928]}]
Train Epoch: 1 [0/10000 (0%)]	Loss: 65.416222	Accuracy: 15.625
Train Epoch: 1 [640/10000 (6%)]	Loss: nan	Accuracy: 7.8125
Train Epoch: 1 [1280/10000 (13%)]	Loss: nan	Accuracy: 6.25
Train Epoch: 1 [1920/10000 (19%)]	Loss: nan	Accuracy: 4.6875
Train Epoch: 1 [2560/10000 (25%)]	Loss: nan	Accuracy: 7.8125
Train Epoch: 1 [3200/10000 (32%)]	Loss: nan	Accuracy: 10.9375
Train Epoch: 1 [3840/10000 (38%)]	Los

KeyboardInterrupt: 

### to use only model file with checkpoint

In [ ]:
from torch import optim

script_params = {
    '--epochs': 5,
    '--batch-size': 64,
    '--test-batch-size': 128,
    '--lr': 0.05
}

net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.05)
estimator = Estimator(use_model=True,model_path="mnist_init_net_checkpoint.pt",network=net,use_optimizer=True,optimizer=optimizer,script_params=script_params,debug=True)
estimator.fit(input_data, input_labels, loss="nll_loss", opt="SGD")

### to use only model file with checkpoint and cuda

In [4]:
from torch import optim

script_params = {
    '--epochs': 5,
    '--batch-size': 64,
    '--test-batch-size': 128,
    '--lr': 0.05
}

net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.05)
estimator = Estimator(use_cuda=True,use_model=True,model_path="mnist_init_net_checkpoint.pt",network=net,use_optimizer=True,optimizer=optimizer,script_params=script_params,debug=True)
estimator.fit(input_data, input_labels, loss="nll_loss", opt="SGD")

CUDA Supported!
PyTorch Model was Loaded.
Model's state_dict:
conv1.weight 	 torch.Size([10, 1, 5, 5])
conv1.bias 	 torch.Size([10])
conv2.weight 	 torch.Size([20, 10, 5, 5])
conv2.bias 	 torch.Size([20])
fc1.weight 	 torch.Size([50, 320])
fc1.bias 	 torch.Size([50])
fc2.weight 	 torch.Size([10, 50])
fc2.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.0010000000474974513, 'momentum': 0.8999999761581421, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140667496647008, 140667496648448, 140665489182464, 140667562701680, 140665489243584, 140667562704160, 140667562704080, 140667562704320]}]
Train Epoch: 1 [0/10000 (0%)]	Loss: 59.207573	Accuracy: 15.625
Train Epoch: 1 [640/10000 (6%)]	Loss: 2.357342	Accuracy: 17.1875
Train Epoch: 1 [1280/10000 (13%)]	Loss: 2.308648	Accuracy: 10.9375
Train Epoch: 1 [1920/10000 (19%)]	Loss: 2.275275	Accuracy: 7.8125
Train Epoch: 1 [2560/10000 (25%)]	Loss: 2.329717	Accuracy: 14.0625
Train Epoch: 1 [3200/10000

# Do Make Horovod Running Test Python File

In [9]:
%%writefile './test_estimator_hvd.py'

from aistudio.estimator import Estimator
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
# import horovod.torch as hvd

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

if __name__ == '__main__':
    # define script parameters
    script_params = {
        '--test-batch-size': 128
    }
    # Data Preparation and Pre-Processing
    download_root = 'train-data'
    mnist_transform=transforms.Compose([
                               transforms.ToTensor()
                               ,transforms.Normalize((0.1307,), (0.3081,))
                               ])
    train_dataset  = datasets.MNIST(download_root, transform=mnist_transform, train=True, download=True)
    input_data = train_dataset.data
    input_data = torch.add(input_data,-0.1307)
    input_data = torch.div(input_data,0.3081)
    input_data = input_data.unsqueeze(1)
    input_labels = train_dataset.targets

    script_params = {
        '--epochs': 5,
        '--batch-size': 64,
        '--test-batch-size': 128,
        '--lr': 0.05
    }
    net = Net()
    optimizer = optim.SGD(net.parameters(), lr=0.05)
    estimator = Estimator(use_cuda=True,use_model=True,model_path="mnist_init_net_checkpoint.pt",network=net,use_optimizer=True,optimizer=optimizer,script_params=script_params)
    estimator.fit(input_data, input_labels, loss="nll_loss", opt="SGD")

Overwriting ./test_estimator_hvd.py
